# BIOINF529 Homework #5 - Winter 2020
This homework is worth **10% of your final grade**.

The exam is due before the next course module begins as enforced by Canvas.

## Coding by Contract
We (the Instructors) promise a fair, impartial, and objective means of grading such that you (the Students) follow the tenets of Coding by Contract:
1. You must not modify/delete any of the existing code in this document (besides the `pass` statements)
* Your functions must use the function signatures as written
* Your functions must return/print the expected results (as written)

If these are followed correctly, your submission should be compatible with the automated testing suite. Therefore, the more tests your code passes, the less scrutiny your code will be under by our review. We do not care *how* you get there, just that you get there *correctly*.

## Submission
Please rename this notebook to **homework5_uniqname.ipynb** for submission. 

For example:
> `homework5_apboyle.ipynb`

We will *only* grade the most recent submission of your exam.

## Late Policy
Each submission will receive a **25%** penalty per day (up to three days) that the assignment is late.

After that, the student will receive a **0** for the homework.

## Academic Honor Code
You may consult with others. However, all answers must be your own and code comparison software will be used to enforce this rule. You are allowed to ask questions at office hours but the answers given will be high-level/conceptual in nature.


---
# 1. Game of Life and Death

In class we built a simnulation of Conway's game of life. Here we will add some complexity to the simulation and include a 'predator' (Green) that relies solely on the existance of feeder cells (Yellow) to survive. 

The feeder cells will follow the same rules as before:
1. Any live cell with fewer than two live neighbours dies, as if by underpopulation.
* Any live cell with two or three live neighbours lives on to the next generation.
* Any live cell with more than three live neighbours dies, as if by overpopulation.
* Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

The predator cells will follow these rules:
1. Any live cell with no neighboring feeder cells dies.
* Any dead cell with at least one neighboring predator cell and more than 3 feeder cells becomes a new predator.

You are expected to implement and will be graded on the following functions:
* `random_grid`
* `count_living_neighbors_feeder`
* `count_living_neighbors_predator`
* `get_neighbors`
* `update_grid`.

This question also has the possibility for **extra credit**. For **10 pts** extra credit, define a new set of rules based on current events: There is current a pandemic driven by the SARS-CoV-2 virus and we are attempting to stop this by shelter in place rules. Using the same agent based modeling system design, seed a random grid with people who are not infected, infected, and immune. Instead of creating new cells in each iteration, update the grid as follows: 
1. All individuals move with probability `r` (this indicates how well the population adheres to shelter in place rules)
* Any non-infected people in contact with an infected becomes infected with probabiliy `p` (this indicates how easy it is to catch the virus from another)
* An infected person becomes immune with probability `q`. Alternatively this can be a set number of movements. (this is to estimate the time until somone is cured)

Your code must be able to be run and generate an animated graphic with this new system.

Implement as defined in the code below and according to the concept of **'coding by contract'** as discussed in class. If your functions do not take as input or provide as output the variables that we define, you will not receive credit.

In [67]:
# You will need these imports
import numpy as np 
import matplotlib.pyplot as plt  
from matplotlib import animation, rc 
from IPython.display import HTML

In [68]:
# Define some global variables - do not change these.
DEAD = 0
ALIVE_FEEDER = 2
ALIVE_PREDATOR = 1

In [69]:
def random_grid(dim, p_alive_feeder, p_alive_predator, seed): 
    """ Initialize a random grid of dim x dim random values
    
    Args:
        dim (int): dimensions for grid (will be square so x = y = dim)
        p_alive_feeder (float): probability of a live feeder cell on grid
        p_alive_predator (float): probability of a live predator cell on grid
        seed (int): seed for np.random.choice
    
    Returns:
        grid (numpy matrix): dim x dim numpy matrix populated with live and dead cells
        
    Example:
    >>> random_grid(4, .3, .3, 1) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    array([[0, 1, 2, 0], ...])
    """

    #main code was taken from Alan's class_25 solutions. I modified the code. 
    
    #take care of the seed first. 
    np.random.seed(seed)
    
    #now get our values.
    values = []
    values.append(ALIVE_FEEDER)
    values.append(DEAD)
    values.append(ALIVE_PREDATOR)
    #order matters here to match the random choice that Alan has in his doc test. 
    #values = [alive_feeder, dead, alive_predator].
    #need to calculate the p_dead probability here.
    p_dead = 1 - (p_alive_feeder + p_alive_predator)
    
    #now randomly select from the values list, with the dims given, and the probabilites. 
    grid = np.random.choice(values, (dim*dim), p = [p_alive_feeder, p_dead, p_alive_predator])
    grid = grid.reshape((dim, dim)) #reshape it to be a dim x dim matrix. 
    
    return grid
    

In [70]:
#doctest for random_grid function. 

#expected output for doctest:
# array([[0, 1, 2, 0], ...])
random_grid(4, .3, .3, 1)

array([[0, 1, 2, 0],
       [2, 2, 2, 0],
       [0, 0, 0, 0],
       [2, 1, 2, 0]])

In [71]:
def count_living_neighbors_feeder(row, column, grid):
    """ Count how many living feeder neighbors exist
    
    Args:
        row (int): row of cell
        column (int): column of cell
        grid (np matrix): matrix of all cells
        
    Returns:
        living_count (int): number of living feeder neighbors
    
    Example:
    >>> count_living_neighbors_feeder(2,2, random_grid(4,.3,.3,1)) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    3
    """
    
    #main code was taken from Alan's class_25 solutions. I modified the code. 
    
    #initialize an empty list to keep track of all the feeders. 
    feeder_list = []
    #calling get_neighbors function here. It returns a list, so I'm setting the list equal to this variable. 
    all_neighbors_list = get_neighbors(row, column, grid) 
    #feeder ID = 2.
    for char in all_neighbors_list:
        if char == ALIVE_FEEDER:
            feeder_list.append(char)
    #count number of ALIVE_FEEDER.
    living_count = len(feeder_list)
    
    return living_count
    

In [72]:
#doctest for count_living_neighbors_feeder function. 

#expected output for doctest:
# 3
count_living_neighbors_feeder(2,2, random_grid(4,.3,.3,1))

3

In [73]:
def count_living_neighbors_predator(row, column, grid):
    """ Count how many living predator neighbors exist
    
    Args:
        row (int): row of cell
        column (int): column of cell
        grid (np matrix): matrix of all cells
        
    Returns:
        living_count (int): number of living predator neighbors
    
    Example:
    >>> count_living_neighbors_predator(2,2, random_grid(4,.3,.3,1)) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    1
    """
    
    #main code was taken from Alan's class_25 solutions. I modified the code. 
    
    #initialize an empty list to keep track of all the predators. 
    predator_list = []
    #calling get_neighbors function here. It returns a list, so I'm setting the list equal to this variable. 
    all_neighbors_list = get_neighbors(row, column, grid)
    #predator ID = 1.
    for char in all_neighbors_list:
        if char == ALIVE_PREDATOR:
            predator_list.append(char)
    #count number of ALIVE_FEEDER.
    living_count = len(predator_list)
    
    return living_count
    

In [74]:
#doctest for count_living_neighbors_predator function. 

#expected output for doctest:
# 1
count_living_neighbors_predator(2,2, random_grid(4,.3,.3,1))

1

In [75]:
def get_neighbors(row, column, grid):
    """ Get the neighboring cells
    excluding any that might be out of bounds
    
    Args:
        row (int): row of cell
        column (int): column of cell
        grid (np matrix): matrix of all cells
        
    Returns:
        neighbors (list): list of adjacent cells
        
    Example:
    >>> get_neighbors(2,2, random_grid(4,.3,.3,1)) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    [2, 2, 0, 0, 0, 1, 2, 0]
    """
    
    #main code was taken from Alan's class_25 solutions. I modified the code. 
    
    #initialize an empty list for neighbors. 
    neighbors = []
    #now we need to iterate through each column and row.
    for r in range(row - 1, row + 2):
        #if we run off the top of the grid. 
        if r < 0: 
            continue
        #if we run off the end of the grid. 
        elif r >= grid.shape[0]: #need index of 0 for rows. 
            continue 
        for col in range(column - 1, column + 2):
            #if we run off the farthest-left of the grid. 
            if col < 0:
                continue 
            #if we run off the farthest-right of the grid. 
            elif col >= grid.shape[1]: #need index of 1 for columns. 
                continue 
            #we don't want to include in our list the point of interest itself. 
            elif r == row and col == column:
                continue 
            #for all other possible conditions:
            else:
                neighbors.append(grid[r][col])
    
    return neighbors
            

In [76]:
#doctest for get_neighbors function. 

#expected output for doctest:
# [2, 2, 0, 0, 0, 1, 2, 0]
get_neighbors(2,2, random_grid(4,.3,.3,1))

[2, 2, 0, 0, 0, 1, 2, 0]

In [77]:
def update_grid(frameNum, img, grid, dim): 
    """ Update grid of cells based on rules:
    
    Any live cell with fewer than two live neighbours dies, as if by underpopulation.
    Any live cell with two or three live neighbours lives on to the next generation.
    Any live cell with more than three live neighbours dies, as if by overpopulation.
    Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

    Args:
        frameNum (int): frame number for the simulation (this is passed by animation)
        img (image object): this is passed by animation
        grid (np array): this is passed by animation
        dim (int): dimensions of grid (this is passed by animation)
        
    This function copies a new Grid over the existing grid for the animation process
    
    """
    
    #main code was taken from Alan's class_25 solutions. I modified the code. 
    
    #need to make a copy to update and continue to reference back to. 
    new_grid = grid.copy()
    
    #iterate through all rows and columns.
    for row in range(dim):
        for col in range(dim):
            #call each count_living_neighbors functions.
            feeder_neighbors = count_living_neighbors_feeder(row, col, grid)
            predator_neighbors = count_living_neighbors_predator(row, col, grid)
            
            #The feeder cells will follow the same rules as before. 
            #if cells are dead.  
            if grid[row][col] == DEAD:
                #FEEDER: Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.
                if feeder_neighbors == 3:
                    new_grid[row][col] = ALIVE_FEEDER
                #PREDATOR: Any dead cell with at least one neighboring predator cell and more than 3 feeder cells becomes a new predator.
                elif predator_neighbors >= 1 and feeder_neighbors > 3:
                    new_grid[row][col] = ALIVE_PREDATOR
            
            #if cells are alive and feeders.
            elif grid[row][col] == ALIVE_FEEDER:
                if feeder_neighbors < 2:
                    new_grid[row][col] = DEAD
                elif feeder_neighbors > 3:
                    new_grid[row][col] = DEAD
                elif feeder_neighbors == 2 or feeder_neighbors == 3:
                    new_grid[row][col] = ALIVE_FEEDER
            
            #if cells are alive and predators.
            elif grid[row][col] == ALIVE_PREDATOR:
                if feeder_neighbors == 0:
                    new_grid[row][col] = DEAD
          
    #update the data. 
    img.set_data(new_grid)
    #take all rows and columns. 
    grid[:] = new_grid[:]
        

In [78]:
# This function does not need to be changed!

def run_simulation(grid_size = 100, p_alive_feeder = .5, p_alive_predator = .05, updateInterval = 500, seed=42): 
    """ Function to run the full simulation
    Each frame is an iteration of the model which calls update_grid
    with the arguments listed in fargs
    
    img should be updated with img.set_data(newGrid) in each iteration of update_grid
    
    """
    
    #main code was taken from Alan's class_25 solutions. 
    
    # declare grid.
    grid = np.array([]) 
    grid = random_grid(grid_size, p_alive_feeder, p_alive_predator, seed=seed) 
  
    # set up animation.
    fig, ax = plt.subplots(figsize=(6,6)) 
    img = ax.imshow(grid) 
    anim = animation.FuncAnimation(fig, update_grid, fargs=(img, grid, grid_size, ), 
                                  frames = 100, 
                                  interval=updateInterval) 
    
    #anim.save('gol.gif', writer='imagemagick', fps=60).
    return HTML(anim.to_html5_video())


In [79]:
%%capture
game_of_life = run_simulation()

In [80]:
game_of_life

Optional Extra credit:


In [ ]:
# Optional extra credit code


---
# 2. Boolean System Simulation

Recall from class that we created a simple boolean system with nodes and update rules and performed simulation on this system, in which all nodes were updated in each time step (i.e. updated using **synchronous** method).

In this section of the test, we will first write a function to read the nodes and rules of the boolean system from an input file, then we will implement simulation on this boolean system using **asynchronous** method for updating nodes.

You are expected to implement the following functions:

* `read_network(filename)`

`Bool_Network.txt` is an example of input file, notice that each line defines the rule for each node in the boolean system,

For example:

`A* = B or C` means that the rule of updation of node `A` is `B or C`.

* `simulation_asynchronous(self, round=1, seed=42)` (this function is inside the `Boolean_Model` class)

We will implement "**Random order asynchronous**" update method here. It is assumed that only one node is updated in each time step, and each node in the network is updated exactly once during each **round** of update. For the purpose of evaluation, you are required to use `random.shuffle()` with the default seed (`seed=42`) to assign an updating order to the nodes of the network at each round of update.

Implement as defined in the code below and according to the concept of **'coding by contract'** as discussed in class. If your functions do not take as input or provide as output the variables that we define, you will not receive credit.

In [89]:
def read_network(filename):
    '''Read a boolean network system from an input file
    
    Args:
        filename (str): input file containing the nodes and rules of a boolean system
        
    Returns:
        rules (dict): A dictionary of rules read from input file
        
    Examples:
        >>> read_network("data/Bool_Network.txt")  #doctest: +NORMALIZE_WHITESPACE
        {'A': 'B or C', 'B': 'A and D', 'C': '(A or B) and not D', 'D': 'not B'}
    '''
    
    #method taken from class_23 solutions 
    
    rules = {}
    file_object = open(filename, "rt")
    #go through each line in the file 
    for line in file_object:
        chars = line.strip().split("=")
        # print(chars)
        # print(chars[0])  #has the "*" and space so need to get rid of both of them. 
        # print(chars[1])  #with this one I see there is a space. so we need to strip it. 
        chars_1 = chars[0].strip("* ") #strip the asterisks
        chars_2 = chars[1].strip() #stripped the extra space. 
        #fill in the empty dict
        rules[chars_1] = chars_2
        
    return rules

In [90]:
#testing read_network function with doctest. 

#expected output: 
#{'A': 'B or C', 'B': 'A and D', 'C': '(A or B) and not D', 'D': 'not B'}
rules = read_network("data/Bool_Network.txt")
rules

{'A': 'B or C', 'B': 'A and D', 'C': '(A or B) and not D', 'D': 'not B'}

In [98]:
import random 

class Boolean_Model:
    ''' Class for boolean models
    
    Class for holding boolean model parameters and to do simulation
    
    Private Attributes:
        _intial (dict): The initial states for nodes
        _rules (dict): The regulatory rules of boolean model 
    '''
    
    def __init__(self, initial, rules):
        self._initial = initial 
        self._rules = rules
    
    @property
    def nodes(self):
        return self._initial.keys()

    def simulation_asynchronous(self, round=1, seed=42):
        '''Function for simulation on boolean model using asynchronous method: an updating order is randomly assigned to nodes
        
        Args:
            round (int): Number of rounds for simulation, each node is updated exactly once during each round of update
            
        Returns:
            results (int, list of dict): A dictionary of nodes containing their state in each time step
            update_order (list of str): Node being updated in each time step
        Examples:
            #Create the boolean model
            >>> initial = {'A': 1, 'B': 1, 'C': 1}
            >>> rules = {'A': 'not B', 'B': 'not C', 'C': 'A'}
            >>> boolean_model = Boolean_Model(initial, rules)
            
            #Simulation using asynchronous method
            >>> boolean_model.simulation_asynchronous() #doctest: +NORMALIZE_WHITESPACE 
            ({'A': [1, 1, 1, 1], 'B': [1, 0, 0, 0], 'C': [1, 1, 1, 1]}, ['B', 'A', 'C'])
        '''
        
        #main code was taken from Alan's class_26 solutions. 
        #take care of seed first. 
        random.seed(seed) 
        #initialize all empty dictionaries and lists.  
        results = {}
        update_order = []
        node_update = {}
        
        #get keys and initial values:
        for node in self.nodes:
            results[node] = [self._initial[node]]
        #update the locals with the initial values. 
        locals().update(self._initial) 
        
        #get all the keys into this separate dict. 
        rules_keys = []
        for rule in rules.keys():
            rules_keys.append(rule) 
        
        #now for every round: 
        for cycle in range(round):
            #now we need to shuffle our rules_keys list for each cycle I want to randomly shuffle the list. 
            random.shuffle(rules_keys)
            #now we need to iterate through each of these chars in this shuffled list. 
            for char in rules_keys:
                #this is what we return in the output. Put each suffled char into this list to be returned in that specific order. 
                update_order.append(char)
                #get all the nodes IDs again in their original order. 
                #put into this empty list. 
                node_id_list = []
                for ID in self.nodes:
                    node_id_list.append(ID)
                
                #now itereate through the node_id list and compare with each node until it matches with the char from the shuffled list. 
                for node_id in node_id_list:
                    if char == node_id:
                        node_update[node_id] = int(eval(self._rules[node_id]))
                        results[node_id].append(node_update[node_id])
                    
                        #when the node_id == char, then we remove it from the node_id list. 
                        #everything else remaining in the node_id list doesn't get a rule applied to it. 
                        #so we need to carry over the most previous value from the most previous column. 
                        node_id_list.remove(node_id)
                        for node_left in node_id_list:
                            #need to grab the most previous value in the column. Use [-1] 
                            node_update[node_left] = results[node_left][-1]
                            results[node_left].append(node_update[node_left])
                        #need to update the locals values. 
                        locals().update(node_update)      
                
        return results, update_order
        

In [99]:
#taken from Alan's class_26 solutions and pasted here to check. 
#expected output from Alan's doctest: 
#({'A': [1, 1, 1, 1], 'B': [1, 0, 0, 0], 'C': [1, 1, 1, 1]}, ['B', 'A', 'C'])

initial = {'A': 1, 'B': 1, 'C': 1}
rules = {'A': 'not B', 'B': 'not C', 'C': 'A'}

boolean_model = Boolean_Model(initial, rules)

In [100]:
results = boolean_model.simulation_asynchronous()
results

({'A': [1, 1, 1, 1], 'B': [1, 0, 0, 0], 'C': [1, 1, 1, 1]}, ['B', 'A', 'C'])

In [95]:
#ANOTHER TEST FOR AN OUTPUT 
#I tested my code again but including the "D" and setting the D's initial value to 1. 
#I wanted to see if it would run and what output would come. 

initial = {'A': 1, 'B': 1, 'C': 1, 'D': 1}
#calling the first function I created for the boolean system stimulation. 
rules = read_network("data/Bool_Network.txt")

boolean_model = Boolean_Model(initial, rules)

In [96]:
#Perform simulation on the boolean model
results = boolean_model.simulation_asynchronous()
results

({'A': [1, 1, 1, 1, 1],
  'B': [1, 1, 1, 1, 1],
  'C': [1, 0, 0, 0, 0],
  'D': [1, 1, 1, 0, 0]},
 ['C', 'B', 'D', 'A'])

## My work for Boolean Problem Below

In [467]:
test = "aaabbbccdf"

In [468]:
test[-1]

'f'

In [69]:
test = ["amelia", "andres", "kira", "alexi"]

In [70]:
len(test)

4

In [44]:
random.shuffle(test)
print(test)

['andres', 'amelia', 'kira', 'alexi']


In [278]:
type(rules)

dict

In [279]:
rules

{'A': 'B or C', 'B': 'A and D', 'C': '(A or B) and not D', 'D': 'not B'}

In [286]:
list(rules)

['A', 'B', 'C', 'D']

In [287]:
rules.keys()

dict_keys(['A', 'B', 'C', 'D'])

In [290]:
test = []
for i in rules.keys():
    test.append(i)
print(test)

['A', 'B', 'C', 'D']


In [294]:
test_02 = []
for i in range(len(test)):
    test_02.append(test[i])
test_02

['A', 'B', 'C', 'D']

In [356]:
test = [1,2,3,4]

In [361]:
for index in range(len(test)):
    print(test[-1])

4
4
4
4


In [363]:
step = []
for i in range(len(test)):
    step.append(test[i])
step

[1, 2, 3, 4]

In [364]:
step = []
for char in test:
    step.append(char)
step

[1, 2, 3, 4]

---
# Testing your code
**Instructions:** If you want a to perform a cursory check of your code (for debugging purposes), do the following:
1. Run all cells that the function(s) you want to test depends on
* Run the cell(s) you want to test
* Run the cell below

**Note:** This tests *only* the test cases that are present in the docstrings. Therefore, a passing test does not mean your program(s) work for *all possible* test cases. It is up to you to determine edge-cases that we may consider during grading.

In [101]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=9)